In [2]:
import imageio
import numpy as np
from pathlib import Path
from omegaconf import OmegaConf
import os
from IPython.display import display, Image, HTML
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
import torch.nn.functional as F
import torch
import yaml
import pandas as pd

def disp_im(im):
    plt.close()
    plt.cla()
    fig = plt.imshow(im)
    plt.close()
    display(fig.figure)


In [3]:
# exps = ["norussian", "noprednorms_nl0", "noprednorms_nl0_conserve_pb0", 'fixedmip128', 'tensorf', 'singlebounce_samen', 'singlebounce']
exps = ["noprednorms_nl0_conserve_pb0", 'fixedmip128', 'tensorf', 'singlebounce_samen', 'interpdiffuse', 'interpdiffuse_flipnorm']
# log_dir = Path("../log") / "norussian"
# log_dir = Path("../log") / "noprednorms_nl0"
rows = []
col_keys = ['psnr', 'ssim', 'l_alex', 'l_vgg', 'envmap_psnr_top', 'envmap_smape_top', 'norm_err', 'envmap_lpips_top', 'envmap_ssim_top']
stats_files = ['stats_augnle2.yaml', 'stats_augnle.yaml', 'stats_augnl.yaml', 'stats_augn.yaml', 'stats_aug.yaml', 'stats.yaml']
col_rows = {k: [] for k in col_keys}
for exp in exps:
    log_dir = Path("../log") / exp
    col_data = {k: dict(exp=exp) for k in col_keys}
    for p in log_dir.glob("*"):
        pano_path = p / "imgs_test_all" / "envmaps" / "pano.exr"
        mapped_pano_path = p / "imgs_test_all" / "envmaps" / "mapped_pano.png"
        
        stat_file = [s for s in stats_files if (p / "imgs_test_all" / s).exists()]
        stat_file = stat_file[0] if len(stat_file) else None
        if stat_file is None:
            print(p)
            continue
        data_path = p / "imgs_test_all" / stat_file
        with data_path.open('r') as f:
            data = yaml.safe_load(f)

        config = OmegaConf.load(p / 'config.yaml')
        dname = Path(config['dataset']['scenedir']).name

        if dname == 'shinyball':
            continue
        data['dname'] = dname
        data['exp'] = exp
        rows.append(data)
        for k in col_keys:
            if k in data:
                col_data[k][dname] = data[k]
    for k in col_keys:
        col_data[k]['mean'] = np.mean([v for k, v in col_data[k].items() if type(v) != str and v != 0])
        col_rows[k].append(col_data[k])

df = pd.DataFrame.from_records(rows)
psnrs = pd.DataFrame.from_dict(col_rows['psnr'])
ssims = pd.DataFrame.from_dict(col_rows['ssim'])
lpips = pd.DataFrame.from_dict(col_rows['l_alex'])
elpips = pd.DataFrame.from_dict(col_rows['envmap_lpips_top'])
epsnrs = pd.DataFrame.from_dict(col_rows['envmap_psnr_top'])
essims = pd.DataFrame.from_dict(col_rows['envmap_ssim_top'])
esmape_top = pd.DataFrame.from_dict(col_rows['envmap_smape_top'])
norm_err = pd.DataFrame.from_dict(col_rows['norm_err'])

shiny_dataset = ["teapot", "toaster", "ball", "car", "coffee", "helmet"]
blender_dataset = [ "chair", "lego", "materials", "mic", "hotdog", "ficus", "drums", "ship" ]
# exp = 'noprednorms_nl0_conserve_pb0'
exps = ['interpdiffuse_flipnorm', 'interpdiffuse', 'fixedmip', 'singlebounce_samen']
# metrics = ['envmap_psnr_top', 'envmap_lpips_top', 'envmap_ssim_top']
for exp in exps:
    print(exp)
    metrics = ['psnr', 'l_alex', 'ssim', 'norm_err']
    for metric in metrics:
        df = pd.DataFrame.from_dict(col_rows[metric])
        vs = []
        for d in blender_dataset:
            vs.append(df.loc[df['exp']==exp][d].item())
        print(f"blender_dataset {metric}: {np.mean(vs)}")
        vs = []
        for d in shiny_dataset:
            vs.append(df.loc[df['exp']==exp][d].item())
        print(f"shiny_dataset {metric}: {np.mean(vs)}")

display(HTML(psnrs.to_html()))
psnrs.to_csv('../psnrs.csv')
ssims.to_csv('../ssims.csv')
essims.to_csv('../essims.csv')
lpips.to_csv('../lpips.csv')
elpips.to_csv('../elpips.csv')
norm_err.to_csv('../norm_err.csv')
epsnrs.to_csv('../epsnrs.csv')
esmape_top.to_csv('../esmapes.csv')
# display(HTML(ssims.to_html()))
# display(HTML(lpips.to_html()))
display(HTML(epsnrs.to_html()))
display(HTML(elpips.to_html()))
# display(HTML(esmape.to_html()))
# display(HTML(esmape_top.to_html()))
display(HTML(norm_err.to_html()))

/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


blender_dataset psnr: 30.70180474590812
shiny_dataset psnr: 33.47674305439264
blender_dataset l_alex: 0.047179666164447555
shiny_dataset l_alex: 0.059328899917115145
blender_dataset ssim: 0.9432490803365056
shiny_dataset ssim: 0.9608011271208893
blender_dataset norm_err: 5.274303495883942
shiny_dataset norm_err: 2.395211244622866


,exp,mic,helmet,toaster,teapot,ficus,materials,chair,coffee,lego,hotdog,ship,drums,car,ball,mean
0,noprednorms_nl0_conserve_pb0,32.932520,34.101595,26.740221,41.176898,29.959617,30.934505,32.519514,30.039538,33.492300,33.605708,27.146749,24.911732,30.718719,36.680686,31.782879
1,fixedmip128,30.590943,34.626613,26.684763,41.893998,29.365387,29.117699,32.231430,28.286087,33.236467,33.544547,26.916537,24.337377,29.304316,31.853813,30.856427
2,tensorf,33.392943,26.781385,20.443238,40.892576,30.149730,25.495765,33.288963,27.920176,34.224168,31.565382,26.561137,24.372476,25.719644,25.860267,29.047704
3,singlebounce_samen,32.943744,34.248888,27.148923,40.365806,29.985411,30.833667,33.256311,29.424212,33.601422,32.404435,27.126084,24.968973,30.498641,35.474419,31.591495
4,interpdiffuse,32.994390,34.183488,26.994174,41.457820,29.606588,30.963801,32.675243,31.131533,33.734459,33.398572,27.395928,24.845457,30.807871,36.285572,31.891064
5,interpdiffuse_flipnorm,33.050238,34.328753,26.860665,41.924415,29.567540,30.906583,32.770268,31.191327,33.675283,33.456589,27.503810,24.862556,30.619858,34.502539,31.801459


,exp,mic,helmet,toaster,teapot,ficus,materials,chair,coffee,lego,hotdog,ship,drums,car,ball,mean
0,noprednorms_nl0_conserve_pb0,12.731996,22.850393,18.324304,17.229118,13.984023,14.763946,13.911304,16.820877,14.955509,17.989155,16.662308,14.502972,15.769844,15.473371,16.140651
1,fixedmip128,7.531878,18.201499,16.090436,13.947121,8.718229,11.989819,10.948197,16.440812,14.735180,15.525177,14.800955,11.726411,13.691510,11.127213,13.248174
2,tensorf,9.559620,14.797166,14.228754,14.158230,14.228754,10.842166,14.228754,15.166527,14.687514,13.704035,13.688950,14.219286,10.831055,10.848503,13.227808
3,singlebounce_samen,9.634925,16.636705,12.769407,10.514705,12.376546,14.030992,13.171896,17.210657,14.173114,14.933650,14.842609,14.253370,11.017060,13.920241,13.534706
4,interpdiffuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,interpdiffuse_flipnorm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,exp,mic,helmet,toaster,teapot,ficus,materials,chair,coffee,lego,hotdog,ship,drums,car,ball,mean
0,noprednorms_nl0_conserve_pb0,1.053192,0.587464,0.925021,0.701885,1.235333,0.574063,1.172260,0.619991,1.106816,1.011655,0.837776,1.052985,0.372662,0.950039,0.871510
1,fixedmip128,0.976127,0.658248,0.523460,0.694601,0.898589,0.856384,0.849245,0.652487,0.804329,0.835386,0.954042,0.846954,0.616711,0.857002,0.787398
2,tensorf,0.859342,0.831942,0.730729,0.753628,0.730729,0.923653,0.730729,0.836490,0.744930,0.767953,0.766810,0.730949,0.921589,0.931516,0.804356
3,singlebounce_samen,1.141398,0.711586,0.703366,0.966065,0.897766,0.605313,1.302290,0.620070,1.102931,1.018524,1.044803,1.107479,0.649698,0.970731,0.917287
4,interpdiffuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,interpdiffuse_flipnorm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,exp,mic,helmet,toaster,teapot,ficus,materials,chair,coffee,lego,hotdog,ship,drums,car,ball,mean
0,noprednorms_nl0_conserve_pb0,0.000000,1.071736,4.003559,0.537483,0.000000,2.231519,0.000000,7.987592,0.000000,0.000000,0.000000,0.000000,1.960036,0.280734,2.581808
1,fixedmip128,0.000000,1.278786,3.999577,0.915768,0.000000,3.303256,0.000000,5.620791,0.000000,0.000000,0.000000,0.000000,2.524800,0.706736,2.621388
2,tensorf,0.000000,9.246346,12.077721,1.232936,0.000000,4.624621,0.000000,11.321970,0.000000,0.000000,0.000000,0.000000,4.549915,15.031007,8.297788
3,singlebounce_samen,0.000000,1.099638,3.399674,0.537704,0.000000,2.401917,0.000000,8.364706,0.000000,0.000000,0.000000,0.000000,1.936939,0.283085,2.574809
4,interpdiffuse,2.337484,1.315025,4.207294,0.528765,4.511727,2.243679,3.294225,6.026980,8.071909,3.983936,12.742601,5.008866,2.033023,0.260180,4.040407
5,interpdiffuse_flipnorm,2.283284,1.239203,4.293120,0.466331,4.406161,2.195185,3.309960,6.042589,7.998552,3.833156,12.867781,5.064971,2.121850,0.306900,4.030646
